# Ransac affine for point cloud alignment
After starfinity for finding ROI and ROI_affine_2masks for affineing ROIs

In this tutorial we will learn to align several point clouds from the ROI using two variants of the ransac affine and ICP algorithm

Prerequistes:
1. starfinity for finding ROI
2. ### ROI_affine_2masks for warping ROIs.Inspect invalid ROIs, extract ROI_SPOTS 

Steps
1. import spots location if using RS-FISH, checked with Napari images
2. load images that the same as the registered or unregistered mask 
3. select each mask, load spot location of ROI (FISHSPOTS,RS-FISH, or starfish), perform
ransac affine, ICP affine for each channels of two rounds
4. ### Inspect invalid ROIs, perform another affine, export spots location of every ROIs. 
5. Visualization of spots of multiple channels and rounds
6. Decode spots with knn neighbor or other 3d seqFISH methods.

For later:Trying to implement other better non-rigid methods, local descriptor methods.

bash
1. for each channel, rounds, and # numbers of ROI (e.g., every 50)

We begin with loading the required modules.

In [1]:
# import pools
import os
import sys
import itertools
from math import pi, sin, cos, sqrt
import numpy as np
import matplotlib.pyplot as plt

import cv2
from matplotlib import cm
from scipy import ndimage
import scipy.io
from skimage import data
from skimage.io import imread, imsave
import pandas as pd
from scipy.spatial import cKDTree
from cv2 import estimateTranslation3D
import tifffile
import seaborn as sns

# import bigstream library
import zarr
import z5py
from bigstream import features
#from bigstream import features1
from bigstream import ransac
from bigstream import affine
#from bigstream import affine1
from bigstream import transform
from fishspot.filter import white_tophat
from fishspot.detect import detect_spots_log

# napari
%gui qt5
import napari
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()

We begin with loading the required modules for ransac.

### Colocalization filter 

In [2]:
def eucldist(coords1, coords2):
    """ Calculates the euclidean distance between 2 lists of coordinates. """
    dist = np.zeros(len(coords1))
    i = 0
    for (x, y) in zip(coords1, coords2):
        p1 = x
        p2 = y
        squared_dist = (p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2
        dist[i] = np.sqrt(squared_dist)
        i = i+1
    return dist

def cloud_distance(spot_fix,spot_mov):
    """compute distance of nearest spot cloud by KNN.
    """
    c0=spot_fix[:,:3].copy()
    c1=spot_mov[:,:3].copy()
    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True)  
    return dist2[:,0]

def colocalization(spot_c0,spot_c1,neighbor_radius):
    #vox=[0.23,0.23,0.38]
    c0=spot_c0[:,:3].copy()
    c1=spot_c1[:,:3].copy()

    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    neighbors = kdtree_c0.query_ball_tree(kdtree_c1, neighbor_radius)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
    ## find the smallest repeated value location, and delete the others
    for i in range(Idx_unique.shape[0]):  # should repeat for only once
    #     print(Idx_unique[i])
        Loc_rep=np.where(idx2==Idx_unique[i])    
    #     print(Loc_rep[0])
        A=dist2[Loc_rep[0],Loc_rep[1]]
        minposition = min(A)
        Loc_min = np.where(A==minposition)[0]
    #     print(Loc_min)
    #     Loc_rep_min=Loc_rep[0][Loc_min[0]]    
        Loc_rep_nouse=np.delete(range(len(Loc_rep[0])),Loc_min)
        dist2[Loc_rep[0][Loc_rep_nouse],Loc_rep[1][Loc_rep_nouse]]=neighbor_radius*2 
    ## find the results that are less than radius; used later column data 
    # when only first row is not exist use latter column, or just dispose it. 
    co_loc=np.where(dist2>neighbor_radius)

    for j in range(dist2.shape[0]):
         if dist2[j,0] < neighbor_radius:
                dist2[j,1] = neighbor_radius*2
    for j in range(dist2.shape[0]):            
         if dist2[j,0] <neighbor_radius or dist2[j,1] <neighbor_radius:
                dist2[j,2] = neighbor_radius*2       
    row_c1 = np.where(dist2<neighbor_radius)
#     print(len(row_c1[0]))

    # lipo spot_c1 is row_c1
    pBind = row_c1[0]
    # print(idx2)
    # lipo spot_c1 is idx2
    pAind = [(idx2[row_c1[0][x], row_c1[1][x]]) for x in range(len(row_c1[0]))]
    lipo_c0 = spot_c0[pAind]
    lipo_c1 = spot_c1[pBind]

#     print(np.unique(pAind).shape) 
    true_pos_c0 = np.delete(spot_c0, pAind, axis=0)
    true_pos_c1 = np.delete(spot_c1, pBind, axis=0) #true

    if spot_c0.shape[0]>0:
        P1 = (lipo_c0.shape[0] / spot_c0.shape[0])*100   # % mov spots from  previous images  /spot_c0.shape
    else:
        P1 = 0
        
    if spot_c1.shape[0]>0:
        P2 = (lipo_c1.shape[0] / spot_c1.shape[0])*100  # % fixed spots can be found in later mov images  /spot_c0.shape
    else:
        P2 = 0
#     print('% P1: ',str(P1) + ';  % P2: ',str(P2)) 
    Dist = np.mean(eucldist(lipo_c0,lipo_c1))

    return lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,Dist,P1,P2

def get_spot_inside(all_spots,segmentation_mini,fix_spacing,ROI_fixed,cc):
    """remove spots outside of a segmenation mask
       change spot location back to segmentation mask in pixels
    """
    fix_spots_mini = all_spots[:,:3] / fix_spacing * [0.5,0.25,0.25] + cc
    spot_mini = np.zeros(len(fix_spots_mini))
    rounded_spot = fix_spots_mini.astype('int') 
    for i in range(0, len(fix_spots_mini)):          
        Coord = rounded_spot[i]
        if Coord[0]<0: Coord[0] = 0
        if Coord[1]<0: Coord[1] = 0
        if Coord[2]<0: Coord[2] = 0
        if Coord[0]>segmentation_mini.shape[0]: Coord[0] = segmentation_mini.shape[0]
        if Coord[1]>segmentation_mini.shape[1]: Coord[1] = segmentation_mini.shape[1]            
        if Coord[2]>segmentation_mini.shape[2]: Coord[2] = segmentation_mini.shape[2]
        idx = segmentation_mini[Coord[0]-1, Coord[1]-1, Coord[2]-1]   # roi id
        if idx == ROI_fixed:
            spot_mini[i] = idx  # add ROI number  
    spots_in = all_spots[np.where(spot_mini==ROI_fixed)]
    spots_out = all_spots[np.where(spot_mini==0)]
    
    spots_in_index = np.where(spot_mini==ROI_fixed)
    ns = spots_in.shape[0]
#     print(f'Image: found {ns} key points inside ROI')
    print(f'Image: found {spots_out.shape[0]} key points outside ROI')
    return spots_in,spots_in_index

## ICP
def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nx3 array of points
        dst: Nx3 array of points
    Output:
        distances: Euclidean distances (errors) of the nearest neighbor
        indecies: dst indecies of the nearest neighbor
    '''
    indecies = np.zeros(src.shape[0], dtype=np.int)
    distances = np.zeros(src.shape[0])
    for i, s in enumerate(src):
        min_dist = np.inf
        for j, d in enumerate(dst):
            dist = np.linalg.norm(s-d)
            # find Nearest dst[j] to src[i]
            if dist < min_dist:
                min_dist = dist
                indecies[i] = j
                distances[i] = dist
    return distances, indecies  

def get_scale(A,B):
    dis_A=get_all_side_length(np.array(A))
    dis_B=get_all_side_length(np.array(B))
    scale = np.abs(dis_B/dis_A)
    mask=np.abs(scale)>0.0001
    scale_sort=np.sort(scale[mask].reshape(-1))
    d_n=len(scale_sort)
    s_mean=scale_sort[int(d_n/4):int(d_n*3/4)].mean() #only use medium data
    return s_mean

def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform between corresponding 3D points A->B
    Input:
      A: Nx3 numpy array of corresponding 3D points
      B: Nx3 numpy array of corresponding 3D points
    Returns:
      T: 4x4 homogeneous transformation matrix
      R: 3x3 rotation matrix
      t: 3x1 column vector
    '''
    assert len(A) == len(B)

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0) 
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    W = np.dot(BB.T, AA)
    U, s, VT = np.linalg.svd(W, full_matrices=True, compute_uv=True)
    R = np.dot(U, VT)

    # special reflection case
    if np.linalg.det(R) < 0:
        VT[2,:] *= -1
        R = np.dot(U, VT)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    #scale 
#     s_mean=get_scale(A,B)
    s_mean=1
    
    # homogeneous transformation
    T = np.identity(4)
#     T[0:3, 0:3] = s_mean * R
    T[0:3, 0:3] = R
    T[0:3, 3] = t
    
    return T, R, t, s_mean
       
def icp(A0, B0,distance_forICP,init_pose = None, max_iterations=200, tolerance=0.0001):
    '''
    The Iterative Closest Point method
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    #  select points
#     distance_forICP = 3

    A,B,_,_,dist,_,_ = colocalization(A0,B0,distance_forICP)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0]))  #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T)  # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T) # FIX
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = np.inf
    distances_iter = np.zeros((max_iterations,1))
    for i in range(max_iterations):
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
        # compute the transformation between the current source and nearest destination points
        T,R,t,s_mean = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  #Sort dst[] by indices
        src = np.dot(T, src)
        # check error
        mean_error = np.sum(distances) / distances.size
        if abs(prev_error-mean_error) < tolerance:
            break
        prev_error = mean_error
        distances_iter[i] = mean_error
        print(f'No.{i} iter icp error:{mean_error}')
    T,R,t,s_mean = best_fit_transform(A, src[0:3,:].T)
    
    src_1 = np.ones((4,A0.shape[0]))  #(4, A.shape[0])
    src_1[0:3,:] = np.copy(A0.T)  # A.T shape (3,20)    
    C = np.dot(T, src_1) # A transform
    C =  C[0:3,:].T      # save file as the same order 
    
    return T, distances, C 

def get_all_side_length(points):
    all_dis=[]
    for i in range(len(points)-1):
        for j in range(i+1,len(points)):
            all_dis.append(points[i]-points[j])
    all_dis=np.array(all_dis)
    return np.linalg.norm(all_dis,axis=1)

def estimate_similarity_transform_3D(A, B):
                 
    assert len(A) == len(B)
    N = A.shape[0];
    mu_A = mean(A, axis=0)
    mu_B = mean(B, axis=0)

    AA = A - tile(mu_A, (N, 1))
    BB = B - tile(mu_B, (N, 1))
    H = transpose(AA) * BB
    U, S, Vt = linalg.svd(H)
    R = Vt.T * U.T
                 
    if linalg.det(R) < 0:
        print ("Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    s_mean=get_scale(A,B)
    t = -s_mean * R * mu_A.T + mu_B.T

    return R, t	,s_mean

In [13]:
##RANSAC modules #c
def get_random_sample(data,n_items: int) -> np.ndarray:
    indices = np.random.choice(len(data), n_items, replace=False)
    return indices

def estimate_n_trials_needed(n_sample_points, inlier_ratio: float = 0.8, probability: float = 0.95) -> int:
    """
    Estimate number of trials needed to select only true inliers with RANSAC.
   
    Parameters
    ----------
    inlier_ratio : number of inliers / number of all points
    n_sample_points :number of selected point in each iteration
    probability : desired probability of success.
    
    Returns
    -------
    Estimated number of trials needed.
    """
    return int(np.log(1 - probability) / np.log(1 - inlier_ratio ** n_sample_points))

#main
def ransac_remove_outliers(
    observation,reference,error_threshold,current_indices,inlier_ratio=0.9,n_sample_points=25,
    inlier_threshold=0,fit_with_final_inliers=True
): 
    '''
    This is to remove outliers after each iteration of ICP

    Parameters
    ----------
    observation - a set of points need to be registered to the ref
    inlier_ratio : number of inliers / number of all point
    error_threshold – threshold value to determine when a data point fits a model
    n_sample_points- number of selected point in each iteration
    fit_with_final_inliers: Fit final model with final inliers solved by the algorithm.
    inlier_threshold – minimum number of data points required to fit the model
  
    max_trials – maximum number of iterations allowed in the algorithm
    
    #not set # d – number of close data points required to assert that a model fits well to data
    
    Returns
    -------
    inlier indices
    '''
    # initialization
    inlier_indices_candidate = None
    inlier_indices = None
    trial_number = 0
    best_score = -np.inf
    best_error = np.inf
    best_model = None
    max_trials = None

    is_termination = None #if iteration should be terminated.
    is_solution_valid = None #if solution valid or not.
    
    max_trials = estimate_n_trials_needed(n_sample_points)
    if max_trials>100:
        max_trials=100
    print(max_trials)
    
    #fit
    for trial in range(max_trials):
        src=observation
        trial_number = trial
        sample = get_random_sample(src.T,n_sample_points) #get data
        srcsample = src[:,sample]
        #distances, indices = nearest_neighbor(srcsample[0:3,:].T,reference[0:3,:].T)
        indices=current_indices[sample]
        currentT,R,t,s_mean = best_fit_transform(srcsample[0:3,:].T,reference[0:3,indices].T)
        
        src= np.dot(currentT,src)
        
        model_distance, indice = nearest_neighbor(src[0:3,:].T, reference[0:3,:].T)
        
        #get inliers
        in_indices = np.where(model_distance <= error_threshold)
        in_indices = np.asarray(in_indices)
        inlier_indices_candidate = in_indices #find inliers
        
        score=inlier_indices_candidate.size
        
        #Verify if the current model is valid
        if score >= inlier_threshold:
            is_valid_solution = True
        else:
            is_valid_solution = False

        if not is_valid_solution:
            continue
        
#         error = np.sum(model_distance[indice]) / model_distance[indice].size
        
#         # Update solution if best so far
#         if (error<best_error):
#             inlier_indices = inlier_indices_candidate.copy()
#             print(f'inliers number:{score},error:{error}')
#             best_error = error
#             best_score=score
#             T_rs=currentT
#             RSdistance=model_distance
            
        p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
        D = p.dot(np.linalg.inv(currentT).T)        ## B->A spots (right) with B->A transform (right)
        C = D[:,:3]
        
        #Test ICP distance change. Minor increased performance
        lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_ICP_RS,_,_ = colocalization(observation,C,neighbor_radius2) 
        if (dist_ICP<best_error):
            inlier_indices = inlier_indices_candidate.copy()
            print(f'inliers number:{score},error:{dist_ICP}')
            best_error = dist_ICP
            best_score = score
            T_rs = currentT
            RSdistance = model_distance
            
#     if fit_with_final_inliers:
#         inliers = observation[0:3,inlier_indices]
#         inliers = np.squeeze(inliers)
#         distances, indices = nearest_neighbor(inliers[0:3,:].T, reference[0:3,:].T)
#         T_rs,_,_,_= best_fit_transform(inliers[0:3,:].T,reference[0:3,indices].T)
#         RSsrc= np.dot(T_rs,observation)
#         RSdistances,_ = nearest_neighbor(RSsrc[0:3,:].T, reference[0:3,:].T)
#         output_error = np.sum(RSdistances) / RSdistances.size
#     if output_error==best_error:
#         print(f'yes match best')
#     else:
#         print(f'output=!best')

    return T_rs,RSdistance,best_error

def RANSAC_ICP(
 A0, B0, distance_forICP,init_pose = None, max_iterations=200, tolerance=0.001
):
    '''
    The Iterative Closest Point with RANSAC
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        distance_forICP
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    # select points
    # distance_forICP = 3
    A,B,_,_,dist,_,_ = colocalization(A0,B0,distance_forICP)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0])) #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T) # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T)
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    error = np.inf
    prev_error_rc =np.inf
    distances_iter = np.zeros((max_iterations,1))
    RSthreshold=1
                
    distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
    
    for i in range(max_iterations):
        # add RANSAC to remove outliers #Shiqi Wang
        #determine threshold
        if i>0:
            RSthreshold=np.quantile(distances, .95)
            #RSthreshold=0.95
            print(f'RSthreshold:{RSthreshold}')

        T1,distancesRS,errorRS = ransac_remove_outliers(src,dst,RSthreshold,indices)
        src1 = np.dot(T1, src)
        distances1, indices1 = nearest_neighbor(src1[0:3,:].T, dst[0:3,:].T)
        # check error of RS
        mean_error = np.sum(distances) / distances.size
        mean_error_RS=np.sum(distances1) / distances1.size
        
        if mean_error_RS<mean_error:
            print(f'RANSACpass,mean_error_RANSAC: {mean_error_RS}')
            src=np.copy(src1)
            distances=distances1
            indices=indices1
            error=mean_error_RS
        else:
            print(f'Ignore RS.')
            error=mean_error
  
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
        # compute the transformation between the current source and nearest destination points
        T0,R,t,s_mean = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  #Sort dst[] by indices
        # update the current source
        # refer to "Introduction to Robotics" Chapter2 P28. Spatial description and transformations

        # check error
        mean_error = np.sum(distances) / distances.size
        print(f'No.{i} iter icp error:{mean_error}')
        
        if mean_error <= error:
            error = mean_error
            distances_iter[i] = mean_error
            src = np.dot(T0, src)
            
        if abs(prev_error_rc-error) < tolerance:
            break
        prev_error_rc = error

    T,R,t,s_mean = best_fit_transform(A, src[0:3,:].T)
    distances_iter=distances_iter[np.argwhere(distances_iter)]
    src_1 = np.ones((4,A.shape[0]))  #(4, A.shape[0])
    src_1[0:3,:] = np.copy(A.T)  # A.T shape (3,20)    
    C = np.dot(T, src_1) # A transform
    C =  C[0:3,:].T     

#     fig=plt.figure(dpi=120,figsize=(2,3))
#     plt.plot(distances_iter[:20])
#     sns.despine() 
#     plt.xlabel('Spots:'+ str(distances_iter[:20].shape[0]))
#     plt.ylabel('Distance/pixel')
#     ave=np.average(distances_iter[:20])
#     plt.title(str(float('%.2f' % ave)))
#     plt.show()
    print(f'distance_iter:{distances_iter}')
    return T,distances,C


def ICP_RANSAC(
    A0, B0, distance_forICP,init_pose = None, max_iterations=200, tolerance=0.001
):
    '''
    The Iterative Closest Point with RANSAC
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        distance_forICP
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    #  select points
#     distance_forICP = 3
    A,B,_,_,dist,_,_ = colocalization(A0,B0,distance_forICP)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0])) #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T) # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T)
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    error = np.inf #best error
    prev_error_rc =np.inf
    distances_iter = np.zeros((max_iterations,1))
    RSthreshold=1
    
    
    for i in range(max_iterations):
        
        
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
        # compute the transformation between the current source and nearest destination points
        T0,R,t,s_mean = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  #Sort dst[] by indices
        # update the current source
        # refer to "Introduction to Robotics" Chapter2 P28. Spatial description and transformations
        
        # check error
        mean_error = np.sum(distances) / distances.size
        print(f'No.{i} iter icp error:{mean_error}')
        
        if mean_error <= error:
            error = mean_error
            distances_iter[i] = mean_error
        
        # add RANSAC to remove outliers #Shiqi Wang
        #determine threshold
        RSthreshold=np.quantile(distances, .95)
        #RSthreshold=0.95
        print(f'RSthreshold:{RSthreshold}')
        
        T1,distancesRS,errorRS = ransac_remove_outliers(src,dst,RSthreshold,indices)
        
        # check error of RS
        print(f'mean_error_RANSAC: {errorRS}')
        if errorRS < error:
            print(f'RANSACpass')
            error = errorRS
            distances_iter[i] = errorRS
            src= np.dot(T1, src)
        else:
            print(f'Ignore RS.')
            src = np.dot(T0, src)
        
        if abs(prev_error_rc-error) < tolerance:
            break
        prev_error_rc = error
            
    T,R,t,s_mean = best_fit_transform(A, src[0:3,:].T)
    distances_iter=distances_iter[np.argwhere(distances_iter)]
    src_1 = np.ones((4,A.shape[0]))  #(4, A.shape[0])
    src_1[0:3,:] = np.copy(A.T)  # A.T shape (3,20)    
    C = np.dot(T, src_1) # A transform
    C =  C[0:3,:].T     

#     fig=plt.figure(dpi=120,figsize=(2,3))
#     plt.plot(distances_iter[:20])
#     sns.despine() 
#     plt.xlabel('Spots:'+ str(distances_iter[:20].shape[0]))
#     plt.ylabel('Distance/pixel')
#     ave=np.average(distances_iter[:20])
#     plt.title(str(float('%.2f' % ave)))
#     plt.show()
    print(f'distance_iter:{distances_iter}')
    return T,distances,C

In [4]:
def ROI_trackaffine_bash(segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,
                         threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,ransac_affine,global_affine_0,
                         inv_affine_0,Transform_0,inv_Transform_0,Chn,global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots):
    """
    default: apply DAPI channel to the FISH channels
    """
    seg_dir = 'E:/Maxprobe_analysis/R2_R1_3tm50/'
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
#     neighbor_radius0 = 12  # 12 : used to find nearest match spots for later local affine
    neighbor_radius1 = 3    # 3 used to find nearest match spots in the validation 
    neighbor_radius2 = neighbor_radius1/3  # /3 used to for ICP
#     image_ransac = 1 # >0 is to get point correspondences from image; or o is to find nearest neighbors
    
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    BB=np.where(segmentation2==ROI_moving)
    dd = [min(BB[0]),min(BB[1]),min(BB[2])]   
    fix_ds = fixed_ROI
    
    # transform DAPI image
    mov_affine_0 = transform.apply_global_affine(fix_ds, moving_ROI,fix_spacing, fix_spacing, global_affine_0,)
    mov_ds = transform.apply_global_affine(fix_ds, mov_affine_0,fix_spacing, fix_spacing,Transform_0,) # inv_Transform_0
#     print(global_affine_0)
#     mov_affine_0 = scipy.ndimage.affine_transform(moving_ROI,np.linalg.inv(global_affine_0)) 
#     mov_ds = scipy.ndimage.affine_transform(mov_affine_0,np.linalg.inv(inv_Transform_0)) 
    
    # if global_detection = 1, Adapt spots of ROI from hAirlocalize(um) or RS-FISH(pixel).
    if global_detection > 0:
        zoom=[2,4,4]
        if hAir == 1:
            # read spot data into memory as numpy arrays  Airlocolize spots data in xyz order: um
            print(f'hAirlocalize points')
            # find specific fixed roi
            spotdir = seg_dir + 'hAir/R2_' + Chn + '_ROI.txt'
            spot_fix=np.loadtxt(spotdir, delimiter=',')
            fixed_spots1 = spot_fix[spot_fix[:,4] == ROI_fixed][:,:3]
    #         print(fixed_spots1)
            # find specific moving roi
            spotdir = seg_dir + 'hAir/R1_' + Chn + '_ROI.txt'
            spot_mov=np.loadtxt(spotdir, delimiter=',')
            moving_spots1 = spot_mov[spot_mov[:,4] == ROI_moving][:,:3]       
            # change to zyx order
            fixed_spots11 = np.transpose(np.array([fixed_spots1[:,2],fixed_spots1[:,1],fixed_spots1[:,0]]))
            moving_spots11 = np.transpose(np.array([moving_spots1[:,2],moving_spots1[:,1],moving_spots1[:,0]]))
            # convert to physical units
            ccc = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
            fix_spots = (fixed_spots11 - ccc * fix_spacing)
            ddd = [min(BB[0])*zoom[0],min(BB[1])*zoom[1],min(BB[2])*zoom[2]]
            mov_spots = (moving_spots11 - ddd * mov_spacing)
           
        else:
        ##RS-FISH
            print(f'RS-FISH points from cluster')
            spotdir = seg_dir + 'RS-FISH/R2_' + Chn + '_ROI.txt'
            spot_fix = np.loadtxt(spotdir, delimiter=',')
            fixed_spots1 = spot_fix[spot_fix[:,4] == ROI_fixed][:,:3]
#             print(fixed_spots1.shape)
            spotdir = seg_dir + 'RS-FISH/R1_' + Chn + '_ROI.txt'
            spot_mov = np.loadtxt(spotdir, delimiter=',')
            moving_spots1 = spot_mov[spot_mov[:,4] == ROI_moving][:,:3] 

            fixed_spots11 = np.transpose(np.array([fixed_spots1[:,2],fixed_spots1[:,1],fixed_spots1[:,0]]))
            moving_spots11 = np.transpose(np.array([moving_spots1[:,2],moving_spots1[:,1],moving_spots1[:,0]]))
            # convert to physical units
            ccc = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
            fix_spots = (fixed_spots11 - ccc * fix_spacing)
            ddd = [min(BB[0])*zoom[0],min(BB[1])*zoom[1],min(BB[2])*zoom[2]]
            mov_spots = (moving_spots11 - ddd * mov_spacing)

    #         print(f'RS-FISH points from FIJI')
    #         spotdir = seg_dir + 'R2_3tm50_1920/ICP_111_C3C1/5_R2_C3_0.txt'
    #         spot_fix=np.loadtxt(spotdir, delimiter='\t')
    #         spotdir = seg_dir + 'R2_3tm50_1920/ICP_111_C3C1/5_R1_C3_1.txt'
    #         spot_mov=np.loadtxt(spotdir, delimiter='\t')
    #         fixed_spots11 = np.transpose(np.array([spot_fix[:,2],spot_fix[:,1],spot_fix[:,0]]))
    #         moving_spots11 = np.transpose(np.array([spot_mov[:,2],spot_mov[:,1],spot_mov[:,0]]))
    #         fix_spots =  fixed_spots11* fix_spacing
    #         mov_spots =  moving_spots11* fix_spacing
        
        mov_spots,_ = spots_random_shuffled(moving_ROI_c2,segmentation2,BB,ROI_moving,mov_spots,Shuffle_spots)
        
        ## apply dapi affine
        mov_spots_x = mov_spots
        points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine_0,np.array([[0,0,0,1]]))) #FIX->FIX_reg
#         warp_spots_1 = points.dot(inv_affine_0.T)
        warp_spots_1 = points.dot(np.linalg.inv(inv_Transform).T)
        
        points = np.append(warp_spots_1[:,:3], np.ones((warp_spots_1.shape[0],1)), axis=1)
#         mov_spots = points.dot(inv_Transform_0.T)
        mov_spots = points.dot(np.linalg.inv(Transform_0.T))
        
        mov_spots = mov_spots[:,:3]
        fix_spots_new = fix_spots
        ns1 = fix_spots.shape[0]
        ns2 = mov_spots.shape[0]    
        print(f'FIXED: found {ns1} key points; MOVING: found {ns2} key points')  
    else: 
        # get spots in pixels
        print('Getting FISH spots from image: disabled now')
         
    # return fix_spots,mov_spots

    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    fix_spots_new,_ = get_spot_inside(fix_spots,segmentation1,fix_spacing,ROI_fixed,cc)
    spot_fix = fix_spots_new
    mov_spots_new = mov_spots
    # print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,_,_,dist1,_,_ = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'pre-ransac_Distance: {np.mean(dist1)}')
    
    ######################### if no spot detected.
    if ns1 >= 10 and ns2 >= 10:
        fix_spots1,mov_spots1,_,_,_,_,_ = colocalization(fix_spots_new,mov_spots_new,neighbor_radius0)  # return in pixel
        print(f'Found {fix_spots1.shape[0]} matched fixed points')
        if ransac_affine == 0:
            global_affine = np.eye(4)[:3]
            inv_affine = np.eye(4)[:3]
        else:
            global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)  
            inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)

        points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine,np.array([[0,0,0,1]]))) #FIX->FIX_reg
#         warp_spots = points.dot(inv_affine.T)
        warp_spots = points.dot(np.linalg.inv(inv_Transform).T)   
        
#         print('colocalization of inside spots after ransac affine') 
        spot_mov = warp_spots
        lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(fix_spots_new,spot_mov,neighbor_radius1)  # return in pixel
        print(f'ransac_Distance: {np.mean(eucldist(lipo_c0,lipo_c1))}')
 
        # image_ransac = 1 is to get point correspondences from image; or 0 is to find nearest neighbors
        if image_ransac != 0 and np.mean(eucldist(lipo_c0,lipo_c1)) > image_ransac:
            
            print("local matching again by correlation of intensities in images")
            fix_spots_img =features1.blob_detection(fix_ds, min_radius, max_radius,
                num_sigma=min(max_radius-min_radius, num_sigma_max),
                threshold=threshold_fixed, exclude_border=cc_radius,)
            mov_spots_img = features1.blob_detection(mov_ds, min_radius, max_radius,
                num_sigma=min(max_radius-min_radius, num_sigma_max),
                threshold=threshold_moving, exclude_border=cc_radius,)
            
            sort_idx = np.argsort(fix_spots_img[:, 3])[::-1]
            fix_spots_img = fix_spots_img[sort_idx, :3][:nspots]
            sort_idx = np.argsort(mov_spots_img[:, 3])[::-1]
            mov_spots_img = mov_spots_img[sort_idx, :3][:nspots]
            # convert to physical units(um)
            fix_spots_img = fix_spots_img* fix_spacing
            mov_spots_img = mov_spots_img* mov_spacing
            fix_spots_img,_ = get_spot_inside(fix_spots_img,segmentation1,fix_spacing,ROI_fixed,cc)
            # get contexts
            ns1 = fix_spots_img.shape[0]
            ns2 = mov_spots_img.shape[0]  
            print(f'FIXED image: found {ns1} key points; MOVING image: found {ns2} key points')
            fix_spots0_img = features.get_spot_context(
                fix_ds, fix_spots_img, fix_spacing, cc_r,)
            mov_spots0_img = features.get_spot_context(
                mov_ds, mov_spots_img, mov_spacing, cc_r,)
            
            correlations = features.pairwise_correlation(
                fix_spots0_img, mov_spots0_img,)
            fix_spots1_img, mov_spots1_img = features.match_points(
                fix_spots0_img, mov_spots0_img,
                correlations, match_threshold,)
            print(f'Found {fix_spots1_img.shape[0]} matched fixed points')
            if ransac_affine == 0:
                global_affine = np.eye(4)[:3]
                inv_affine = np.eye(4)[:3]
            else:
                global_affine = ransac_align_points(fix_spots1_img, mov_spots1_img, align_threshold,)  
                inv_affine = ransac_align_points(mov_spots1_img, fix_spots1_img, align_threshold,)
            
            warp_spots = []
            points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
            inv_Transform = np.row_stack((global_affine,np.array([[0,0,0,1]]))) #FIX->FIX_reg
    #         warp_spots = points.dot(inv_affine.T)
            warp_spots = points.dot(np.linalg.inv(inv_Transform).T)   
            
    #         print('colocalization of inside spots after ransac affine')
            spot_mov = []
            spot_mov = warp_spots
            lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(fix_spots_new,spot_mov,neighbor_radius1)  # return in pixel
            print(f'ransac_Distance_again: {np.mean(eucldist(lipo_c0,lipo_c1))}')
                
#         print('registration of all inside spots with ICP')
#         A = spot_fix[:,:3]
#         B = spot_mov[:,:3]
# #         Transform, distances1,C = icp(A, B,neighbor_radius2)
#         inv_Transform, distances2,C = icp(B, A,neighbor_radius2)
#     #     print (Transform)
#         # functions for applying transforms are in bigstream.transform. apply the ICP affine to the moved image
#         ICP_affine = transform.apply_global_affine(
#             fix_ds, mov_affine,
#             fix_spacing, fix_spacing,
#             Transform,)
#         #     print (inv_Transform)
#         p = np.append(B, np.ones((B.shape[0],1)), axis=1)
#         C = p.dot(inv_Transform.T)
        
        A = spot_fix[:,:3]
        B = spot_mov[:,:3]
        return A, B,neighbor_radius2
    
#         Transform,_,_ = icp(A, B,neighbor_radius2)
        
    
#         # A->B 
#         inv_Transform, distances2, _ = icp(B, A,neighbor_radius2) # B->A
#         Transform[np.isnan(Transform)] = 0
#         inv_Transform[np.isnan(inv_Transform)] = 0
#         p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
#         D = p.dot(np.linalg.inv(Transform).T)        ## B->A spots (right) with B->A transform (right)
# #         ICP_affine = scipy.ndimage.affine_transform(mov_affine,np.linalg.inv(inv_Transform)) 
#         C = D[:,:3]
        
#         # Test ICP distance change. Minor increased performance
#         warp_spots_new,_ = get_spot_inside(C,segmentation1,fix_spacing,ROI_fixed,cc)
#         spot_mov = warp_spots_new
#         lipo_c0,lipo_c1,_,_,dist3,P1,P2 = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
#         print('% P1: ',str(P1) + ';  % P2: ',str(P2))
#         pair_dist = eucldist(lipo_c0,lipo_c1)
#         print(f'ICP_Distance: {np.mean(pair_dist)}')
        
#         # functions for images
# #         if image_ransac > 0:
#     else:
#         print(f'Not found enough matched fixed points')
#         global_affine = np.eye(4)[:3]    
#         inv_affine = np.eye(4)[:3]
#         warp_spots = mov_spots
#         lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(spot_fix,warp_spots,neighbor_radius1)  # return in pixel
# #         print(f'Distance: {np.mean(cloud_distance(spot_fix,spot_mov))}')
#         Transform = np.eye(4)[:4]
#         inv_Transform = np.eye(4)[:4]
#         C = warp_spots[:,:3]
#         warp_spots_new,_ = get_spot_inside(C,segmentation1,fix_spacing,ROI_fixed,cc)
#         spot_mov = warp_spots_new
#         lipo_c0,lipo_c1,_,_,dist3,P1,P2 = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
#         print('% P1: ',str(P1) + ';  % P2: ',str(P2)) 
#         pair_dist = eucldist(lipo_c0,lipo_c1)
#         print(f'ICP_Distance: {np.mean(pair_dist)}')
    
#     if Chn == "C3":
#         # bleedthrough correction 
#         lo = np.percentile(np.ndarray.flatten(moving_ROI_c2),99.5) #Estimate fluorescence intensity of bright DAPI signals
#         bg_dapi=np.percentile(np.ndarray.flatten(moving_ROI_c2[moving_ROI_c2!=0]),1) 
#         bg_data=np.percentile(np.ndarray.flatten(moving_ROI[moving_ROI!=0]),1) #Estimate background in AF546 channel
#         dapi_factor=np.median((moving_ROI[moving_ROI_c2>lo] - bg_data)/(moving_ROI_c2[moving_ROI_c2>lo] - bg_dapi)) #Estimate % of signal bleedthrough
#         moving_ROI_new  = np.maximum(0, moving_ROI - bg_data - dapi_factor * (moving_ROI_c2 - bg_dapi)).astype('float32')#subtract background and bleedthrough
#         mov_affine_0 = transform.apply_global_affine(fix_ds, moving_ROI_new,fix_spacing, fix_spacing,global_affine_0,)
#         mov_ds = transform.apply_global_affine(fix_ds, mov_affine_0,fix_spacing,fix_spacing,Transform_0,)  
        
# #     mov_affine = scipy.ndimage.affine_transform(mov_ds,np.linalg.inv(global_affine))
# #     ICP_affine = scipy.ndimage.affine_transform(mov_affine,np.linalg.inv(inv_Transform)) 
#     mov_affine = transform.apply_global_affine(fix_ds, mov_ds, fix_spacing, fix_spacing,global_affine,)
#     ICP_affine = transform.apply_global_affine(fix_ds, mov_affine, fix_spacing, fix_spacing,Transform,) 

#     return P1,P2,dist1,dist2,pair_dist,spot_fix,mov_spots,warp_spots_new,ICP_affine,mov_affine,cc  

In [5]:
def ROI_trackaffine_cca_bash(segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,
                         threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,ransac_affine,
                         global_affine_fix,inv_affine_fix,Transform_fix,global_affine_mov,inv_affine_mov,
                        Transform_mov,global_affine_0,inv_affine_0,Transform_0,inv_Transform_0,Chn,global_detection,
                         image_ransac,hAir,neighbor_radius0,Shuffle_spots):
    """
    1.Transform the affine matrix to correct chromatic aberration of two tracks for each rounds. fix and moving. C0 C1
    points: points.dot(inv_Transform_0.T)
    images: transform.apply_global_affine
    2.Transform the DAPI affine matrix: points and images. 
    3.Do ROI-affine
    
    default: apply DAPI channel affine matrix to the FISH channels
    """
    seg_dir = 'E:/Maxprobe_analysis/R2_R1_3tm50/'
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
#     neighbor_radius0 = 12  # 12 : used to find nearest match spots for later local affine
    neighbor_radius1 = 3    # 3 used to find nearest match spots in the validation 
    neighbor_radius2 = neighbor_radius1/3  # /3 used to for ICP
#     image_ransac = 1 # >0 is to get point correspondences from image; or o is to find nearest neighbors
    
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    BB=np.where(segmentation2==ROI_moving)
    dd = [min(BB[0]),min(BB[1]),min(BB[2])]
    
    
    # transform cca for fixed images
#     fix_ds = fixed_ROI
    fix_affine_0 = transform.apply_global_affine(fixed_ROI,fixed_ROI,fix_spacing, fix_spacing, global_affine_fix,)
    fix_ds = transform.apply_global_affine(fixed_ROI, fix_affine_0,fix_spacing, fix_spacing,Transform_fix,)
#     fix_affine_0  = scipy.ndimage.affine_transform(fixed_ROI,np.linalg.inv(global_affine_fix)) 
#     fix_ds = scipy.ndimage.affine_transform(fix_affine_0 ,np.linalg.inv(inv_Transform_fix)) 
    
    # transform cca and DAPI affine for moving images
    mov_affine_cca = transform.apply_global_affine(moving_ROI,moving_ROI,fix_spacing, fix_spacing, global_affine_mov,)
    mov_ds_cca = transform.apply_global_affine(moving_ROI, mov_affine_cca,fix_spacing, fix_spacing,Transform_mov,)
#     mov_affine_cca  = scipy.ndimage.affine_transform(moving_ROI,np.linalg.inv(global_affine_mov)) 
#     mov_ds_cca = scipy.ndimage.affine_transform(mov_affine_cca,np.linalg.inv(inv_Transform_mov))     
    # need to export and check colocalization
    mov_affine_0 = transform.apply_global_affine(fix_ds, mov_ds_cca,fix_spacing, fix_spacing, global_affine_0,)
    mov_ds = transform.apply_global_affine(fix_ds, mov_affine_0,fix_spacing, fix_spacing,Transform_0,)
#     mov_affine_0  = scipy.ndimage.affine_transform(mov_ds_cca,np.linalg.inv(global_affine_0)) 
#     mov_ds = scipy.ndimage.affine_transform(mov_affine_0,np.linalg.inv(Transform_0)) 
    
    # if global_detection = 1, Adapt spots of ROI from hAirlocalize(um) or RS-FISH(pixel).
    if global_detection > 0:
        zoom=[2,4,4]
        if hAir == 1:
            # read spot data into memory as numpy arrays  Airlocolize spots data in xyz order: um
            print(f'hAirlocalize points')
            spotdir1 = seg_dir + 'hAir/R2_' + Chn + '_ROI.txt'
            spotdir2 = seg_dir + 'hAir/R1_' + Chn + '_ROI.txt'
        else:
            ##RS-FISH
            print(f'RS-FISH points from cluster')
            spotdir1 = seg_dir + 'RS-FISH/R2_' + Chn + '_ROI.txt'
            spotdir2 = seg_dir + 'RS-FISH/R1_' + Chn + '_ROI.txt'

        spot_fix=np.loadtxt(spotdir1, delimiter=',')
        fixed_spots1 = spot_fix[spot_fix[:,4] == ROI_fixed][:,:3]
        # find specific moving roi
        spot_mov=np.loadtxt(spotdir2, delimiter=',')
        moving_spots1 = spot_mov[spot_mov[:,4] == ROI_moving][:,:3]       
        # change to zyx order
        fixed_spots11 = np.transpose(np.array([fixed_spots1[:,2],fixed_spots1[:,1],fixed_spots1[:,0]]))
        moving_spots11 = np.transpose(np.array([moving_spots1[:,2],moving_spots1[:,1],moving_spots1[:,0]]))
        # convert to physical units
        ccc = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
        fix_spots = (fixed_spots11 - ccc * fix_spacing)
        ddd = [min(BB[0])*zoom[0],min(BB[1])*zoom[1],min(BB[2])*zoom[2]]
        mov_spots = (moving_spots11 - ddd * mov_spacing)
        
        ########################################################################
        ## apply cca affine for fix spots
        
        points1 = np.append(fix_spots, np.ones((fix_spots.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine_fix,np.array([[0,0,0,1]]))) #FIX->FIX_reg
        warp_spots_1 = points1.dot(np.linalg.inv(inv_Transform).T)[:,:3] 
        points2 = np.append(warp_spots_1, np.ones((warp_spots_1.shape[0],1)), axis=1)
        fix_spots = points2.dot(np.linalg.inv(Transform_fix).T)
        fix_spots = fix_spots[:,:3]
        
        ## apply cca affine for mov spots
        points2 = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine_mov,np.array([[0,0,0,1]]))) #FIX->FIX_reg
        warp_spots_2 = points2.dot(np.linalg.inv(inv_Transform).T)[:,:3]
        points2 = np.append(warp_spots_2, np.ones((warp_spots_2.shape[0],1)), axis=1)
        mov_spots_x = points2.dot(np.linalg.inv(Transform_mov).T)
        mov_spots_x = mov_spots_x[:,:3]
        ## apply dapi affine for mov 
        points3 = np.append(mov_spots_x, np.ones((mov_spots_x.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine_0,np.array([[0,0,0,1]]))) #FIX->FIX_reg
        warp_spots_3 = points3.dot(np.linalg.inv(inv_Transform).T)[:,:3]
        points3 = np.append(warp_spots_3, np.ones((warp_spots_3.shape[0],1)), axis=1)
        mov_spots = points3.dot(np.linalg.inv(Transform_0.T))
        mov_spots = mov_spots[:,:3]
        
        ns1 = fix_spots.shape[0]
        ns2 = mov_spots.shape[0]    
        print(f'FIXED: found {ns1} key points; MOVING: found {ns2} key points')  
    else: 
        # get spots in pixels
        print('Getting FISH spots from image: disabled now')
    
    mov_spots,_ = spots_random_shuffled(moving_ROI_c2,segmentation2,BB,ROI_moving,mov_spots,Shuffle_spots)
    
    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    fix_spots_new,_ = get_spot_inside(fix_spots,segmentation1,fix_spacing,ROI_fixed,cc)
    spot_fix = fix_spots_new
    mov_spots_new = mov_spots
    # print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,_,_,dist1,_,_ = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'pre-ransac_Distance: {np.mean(dist1)}')
    
    ######################### if no spot detected.
    if ns1 >= 10 and ns2 >= 10:
        fix_spots1,mov_spots1,_,_,_,_,_ = colocalization(fix_spots_new,mov_spots_new,neighbor_radius0)  # return in pixel
        print(f'Found {fix_spots1.shape[0]} matched fixed points')
        if ransac_affine == 0:
            global_affine = np.eye(4)[:3]
            inv_affine = np.eye(4)[:3]
        else:
            global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)  
            inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)

        points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
        inv_Transform = np.row_stack((global_affine,np.array([[0,0,0,1]]))) #FIX->FIX_reg  
        warp_spots = points.dot(np.linalg.inv(inv_Transform).T)
#         print('colocalization of inside spots after ransac affine') 
        spot_mov = warp_spots
        lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(fix_spots_new,spot_mov,neighbor_radius1)  # return in pixel
        print(f'ransac_Distance: {np.mean(eucldist(lipo_c0,lipo_c1))}')
 
        # image_ransac = 1 is to get point correspondences from image; or 0 is to find nearest neighbors
        if image_ransac != 0 and np.mean(eucldist(lipo_c0,lipo_c1)) > image_ransac:
            
            print("local matching again by correlation of intensities in images")
            fix_spots_img =features1.blob_detection(fix_ds, min_radius, max_radius,
                num_sigma=min(max_radius-min_radius, num_sigma_max),
                threshold=threshold_fixed, exclude_border=cc_radius,)
            mov_spots_img = features1.blob_detection(mov_ds, min_radius, max_radius,
                num_sigma=min(max_radius-min_radius, num_sigma_max),
                threshold=threshold_moving, exclude_border=cc_radius,)
            
            sort_idx = np.argsort(fix_spots_img[:, 3])[::-1]
            fix_spots_img = fix_spots_img[sort_idx, :3][:nspots]
            sort_idx = np.argsort(mov_spots_img[:, 3])[::-1]
            mov_spots_img = mov_spots_img[sort_idx, :3][:nspots]
            # convert to physical units(um)
            fix_spots_img = fix_spots_img* fix_spacing
            mov_spots_img = mov_spots_img* mov_spacing
            fix_spots_img,_ = get_spot_inside(fix_spots_img,segmentation1,fix_spacing,ROI_fixed,cc)
#             mov_spots_new1 = get_spot_inside(mov_spots,segmentation2,fix_spacing,ROI_fixed,cc) ## avoid no features
            # get contexts
            ns1 = fix_spots_img.shape[0]
            ns2 = mov_spots_img.shape[0]  
            print(f'FIXED image: found {ns1} key points; MOVING image: found {ns2} key points')
            fix_spots0_img = features.get_spot_context(
                fix_ds, fix_spots_img, fix_spacing, cc_r,)
            mov_spots0_img = features.get_spot_context(
                mov_ds, mov_spots_img, mov_spacing, cc_r,)
            
            correlations = features.pairwise_correlation(
                fix_spots0_img, mov_spots0_img,)
            fix_spots1_img, mov_spots1_img = features.match_points(
                fix_spots0_img, mov_spots0_img,
                correlations, match_threshold,)
            print(f'Found {fix_spots1_img.shape[0]} matched fixed points')
            if ransac_affine == 0:
                global_affine = np.eye(4)[:3]
                inv_affine = np.eye(4)[:3]
            else:
                global_affine = ransac_align_points(fix_spots1_img, mov_spots1_img, align_threshold,)  
                inv_affine = ransac_align_points(mov_spots1_img, fix_spots1_img, align_threshold,)
            
            points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
            inv_Transform = np.row_stack((global_affine,np.array([[0,0,0,1]]))) #FIX->FIX_reg  
            warp_spots = points.dot(np.linalg.inv(inv_Transform).T)
    #         print('colocalization of inside spots after ransac affine') 
            spot_mov = warp_spots
            lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(fix_spots_new,spot_mov,neighbor_radius1)  # return in pixel
            print(f'ransac_Distance_again: {np.mean(eucldist(lipo_c0,lipo_c1))}')
                
        A = fix_spots_new[:,:3]
        B = spot_mov[:,:3]
        Transform,_,_ = icp(A, B,neighbor_radius2)  # A->B 
        return A, B,neighbor_radius2
    
#         inv_Transform, distances2, _ = icp(B, A,neighbor_radius2) # B->A 
#         p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
#         D = p.dot(np.linalg.inv(Transform).T)        ## B->A spots (right) with B->A transform (right)
# #         ICP_affine = scipy.ndimage.affine_transform(mov_affine,np.linalg.inv(inv_Transform)) 
#         C = D[:,:3]
        
#         # Test ICP distance change. Minor increased performance
#         warp_spots_new,_ = get_spot_inside(C,segmentation1,fix_spacing,ROI_fixed,cc)
#         spot_mov = warp_spots_new
#         lipo_c0,lipo_c1,_,_,dist3,P1,P2 = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
#         print('% P1: ',str(P1) + ';  % P2: ',str(P2))
#         pair_dist = eucldist(lipo_c0,lipo_c1)
#         print(f'ICP_Distance: {np.mean(pair_dist)}')
        
#         # functions for images
# #         if image_ransac > 0:
#     else:
#         print(f'Not found enough matched fixed points')
#         global_affine = np.eye(4)[:3]    
#         inv_affine = np.eye(4)[:3]
#         warp_spots = mov_spots
#         lipo_c0,lipo_c1,_,_,dist2,_,_ = colocalization(spot_fix,warp_spots,neighbor_radius1)  # return in pixel
# #         print(f'Distance: {np.mean(cloud_distance(spot_fix,spot_mov))}')
#         Transform = np.eye(4)[:4]
#         inv_Transform = np.eye(4)[:4]
#         C = warp_spots[:,:3]
#         warp_spots_new,_ = get_spot_inside(C,segmentation1,fix_spacing,ROI_fixed,cc)
#         spot_mov = warp_spots_new
#         lipo_c0,lipo_c1,_,_,dist3,P1,P2 = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
#         print('% P1: ',str(P1) + ';  % P2: ',str(P2)) 
#         pair_dist = eucldist(lipo_c0,lipo_c1)
#         print(f'ICP_Distance: {np.mean(pair_dist)}')
    
#     if Chn == "C3":
#         # bleedthrough correction 
#         lo = np.percentile(np.ndarray.flatten(moving_ROI_c2),99.5) #Estimate fluorescence intensity of bright DAPI signals
#         bg_dapi=np.percentile(np.ndarray.flatten(moving_ROI_c2[moving_ROI_c2!=0]),1) 
#         bg_data=np.percentile(np.ndarray.flatten(moving_ROI[moving_ROI!=0]),1) #Estimate background in AF546 channel
#         dapi_factor=np.median((moving_ROI[moving_ROI_c2>lo] - bg_data)/(moving_ROI_c2[moving_ROI_c2>lo] - bg_dapi)) #Estimate % of signal bleedthrough
#         moving_ROI_new  = np.maximum(0, moving_ROI - bg_data - dapi_factor * (moving_ROI_c2 - bg_dapi)).astype('float32')#subtract background and bleedthrough
#         mov_affine_0 = transform.apply_global_affine(fix_ds, moving_ROI_new,fix_spacing, fix_spacing,global_affine_0,)
#         mov_ds = transform.apply_global_affine(fix_ds, mov_affine_0,fix_spacing,fix_spacing,Transform_0,)
        
# #     mov_affine = scipy.ndimage.affine_transform(mov_ds,np.linalg.inv(global_affine)) 
#     mov_affine = transform.apply_global_affine(fix_ds, mov_ds, fix_spacing, fix_spacing,global_affine,)
#     ICP_affine = transform.apply_global_affine(fix_ds, mov_affine, fix_spacing, fix_spacing,Transform,) 
# #     ICP_affine = scipy.ndimage.affine_transform(mov_affine,np.linalg.inv(invTransform)) 
    
#     return P1,P2,dist1,dist2,pair_dist,spot_fix,mov_spots,warp_spots_new,fix_ds,ICP_affine,mov_affine,cc  

## Preparing the raw images and segmented ROIs (registered)
We load the images of 2 channels of two rounds

In [6]:
%%time
# # file paths to bigstrem REGISTERED data N5 files(include deform)
# create Zarr file object using N5Stores
seg_dir='E:/Maxprobe_analysis/R2_R1_3tm50/' # Analyze the below images ## image is in zyx or
im_fixed = z5py.File(seg_dir + 'R2_3tm50_1920/stitching/export.n5', use_zarr_format=False)
im_moving = z5py.File(seg_dir + 'R1_3tm50_1920/stitching/export.n5', use_zarr_format=False)

# # UNREGISTERED of two rounds
segmentation1=imread(seg_dir + 'R2_filtered_mask.tif')
# # Registered and enlarged ROI
segmentation2=imread(seg_dir + 'R1_filtered_mask.tif')
# viewer.add_image(segmentation1,colormap='green',blending='additive') #load image data into napari
# viewer.add_image(segmentation2,colormap='red',blending='additive') #load image data into napari

RuntimeError: 

### Assign_roi_images
#### need to load big raw images for each tile if using distributed computing

In [7]:
%%time 
# t = time()
En_pixels = 0
tile_number = 5
out_dir = seg_dir +'R1_3tm50_1920/registration/python/tiles/'
fix_path = seg_dir + 'R2_3tm50_1920/ROI/'
roi_dir = seg_dir + 'allroi_matched.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
# assign_roi_images(tile_number,fix_path,out_dir,seg_dir,roi_dir,segmentation1,segmentation2,En_pixels)
# print('tictoc：%.2fs' % (time() - t))

Wall time: 0 ns


### ROI_affine with image assisted cca on the true or shuffled points

In [8]:
%%time  
#time: 60 images, 60 s for 4 images (c0-c3) if apply image ransac,40 s for 4 images (c0-c3) if not.
tile_number = 5 #35
seg_dir='E:/Maxprobe_analysis/R2_R1_3tm50/'
roi_dir = seg_dir + 'allroi_matched.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
df_allroi = pd.read_csv(roi_dir)
spot_fix_c3_all = np.zeros((1, 4))
warp_spots_c3_all = np.zeros((1, 4))
spot_fix_c1_all = np.zeros((1, 4))
warp_spots_c1_all = np.zeros((1, 4))
spot_fix_c0_all = np.zeros((1, 4))
warp_spots_c0_all = np.zeros((1, 4))
fix_spacing=np.array([0.42,0.23,0.23])
j=0
Dist_sum_1 = np.zeros((len(df_allroi['fix']), 14)) # directory to file containing the registration stats
affine_borrow = np.zeros((len(df_allroi['fix']), 6))
# Dist_sum_1 = np.zeros((len(gad1_list), 14)) # when apply for GAD1 cells.
# affine_borrow = np.zeros((len(gad1_list), 6))
Shuffle_spots = 0

for i in range(0, tile_number): # distributed
#     i = 0
#     seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/segmentation/'
#     segmentation1=imread(seg_dir+'mask_all_R2.tif')
#     seg_dir='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
#     segmentation2=imread(seg_dir+'mask_all_R1.tif') # Before bigstream
    roi_dir = seg_dir + 'allroi_matched.csv'       # directory to file containing the ROI correspondence
    df_allroi = pd.read_csv(roi_dir)
    fix3_path = seg_dir + 'R2_3tm50_1920/ROI/'
    ROI_RGB_path = seg_dir + 'R2_3tm50_1920/ROI_RGB/'
    if not os.path.exists(ROI_RGB_path): os.mkdir(ROI_RGB_path)
    roi_dir = fix3_path + str(i) + '/ROI_id.csv'   # directory to file containing the ROI correspondence of single tile
    df_tile_roi = pd.read_csv(roi_dir)
    ROI_id = df_tile_roi['fix'].astype(int)
#     ROI_id = ROI_all[roi_inc*i:(i+1)*roi_inc]
    
    for aa in ROI_id:
#         if aa in gad1_list: # < 1106
        if aa == 5: #81 c2 529 for shuffle?
            roi_dir = fix3_path + str(i) + '/R2_' + str(aa) + '_C2.tif'
            fixed_ROI_c2 = imread(roi_dir)
#             ROI_moving = df_allroi.loc[df_allroi['fix'].astype(int) == aa]['mov'].values[0].astype(int) #379
            ROI_moving = aa
            print('ROI_fix: ' + str(aa) + '; ROI_moving: ' + str(ROI_moving))
            roi_dir = fix3_path + str(i) + '/R1_' + str(ROI_moving) + '_C2.tif'
            moving_ROI_c2 = imread(roi_dir)

            #for dapi: ~50 blobs can be detected
            print("DAPI")
            threshold_fixed=0.001/50 #/50
            threshold_moving=0.001/5 #/5
            cc_r=12 
            match_threshold=0.6  #0.7
            align_threshold=0.5
            Global_detection = 0
            image_ransac = 1 # 1 is to get point correspondences from image; or 0 is to use nearest neighbors for local ransac
            P1_C2,P2_C2,_,_,distances_2,global_affine2,inv_affine2,Transform2,inv_Transform2,fix_spots_2,mov_spots_2,warp_spots_new_2,ICP_affine_2,cc_2 = ROI_affine_bash(
            segmentation1,segmentation2,aa,ROI_moving,fixed_ROI_c2,moving_ROI_c2,threshold_fixed,threshold_moving,
            cc_r,match_threshold,align_threshold,Global_detection,image_ransac)
            
            a_rgb = np.zeros(fixed_ROI_c2.shape + (4,))
            a_rgb[..., 0] = fixed_ROI_c2.astype(np.uint16)
            a_rgb[..., 1] = ICP_affine_2.astype(np.uint16)
            
            Global_detection = 1
            ransac_affine = 1 ## 1 is to apply local ransac affine (not use image) before ICP,0 is used orginal point cloud for ICP.
#             image_ransac = 1 # > 0 is to get point correspondences from image after using nearest neighbors ; or 0 is to use nearest neighbors for local ransac
            hAir = 0 # 0 is RS-FISH; 1 is for hAirlocalize
            # find affine for cca
            spotdir = seg_dir + 'R2ROI_points_fix.txt'
            Rounds = ["R2","R1"]
            global_affine_fix,inv_affine_fix,Transform_fix,global_affine_mov,inv_affine_mov,Transform_mov,affine_borrow_idx = cca(
                seg_dir,spotdir,aa,Rounds) 

            for Chn in ["C3"]: # ["C1","C3","C0"]
#                 Chn = "C3"
                print(Chn)
                roi_dir = fix3_path + str(i) + '/R2_' + str(aa) + '_'+ Chn + '.tif'
                fixed_ROI = imread(roi_dir)
                roi_dir = fix3_path + str(i) + '/R1_' + str(ROI_moving) + '_'+ Chn + '.tif'
                moving_ROI = imread(roi_dir)
                # save rgb images 
                if Chn == "C1":
                    #for c1
                    threshold_fixed=0.001/20 #/20
                    threshold_moving=0.001/2 #/2
                    match_threshold=0.5  #0.5
                    align_threshold=0.5  # 0.5
                    neighbor_radius0 = 12
                    image_ransac = 1 # number is specific distance threshold
                    P1_C1,P2_C1,_,_,pair_dist_1, fix_spots_1,mov_spots_1,warp_spots_new_1,fixed_ROI_c1,ICP_affine_1,moving_affine_1,cc_1 = ROI_trackaffine_cca_bash(
                        segmentation1,segmentation2,aa,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,threshold_fixed,threshold_moving,
                        cc_r,match_threshold,align_threshold,ransac_affine,global_affine_fix,inv_affine_fix,Transform_fix,
                        global_affine_mov,inv_affine_mov,Transform_mov,global_affine2,inv_affine2,Transform2,inv_Transform2,
                        Chn,Global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots)
                    
                    distances_1 = np.mean(pair_dist_1)                   
                    a_rgb[..., 2] = fixed_ROI_c1.astype(np.uint16)
                    a_rgb[..., 3] = ICP_affine_1.astype(np.uint16)
#                     a_rgb[..., 3] = moving_affine_1.astype(np.uint16)
                    roi_dir = ROI_RGB_path + '/'+ str(aa) + '_' + 'C2' + Chn +'.tif'
                    imsave(roi_dir, a_rgb)
                    a_rgb = []

                elif Chn == "C3":
                    #for c3
                    
                    threshold_fixed=0.001/8 #/4
                    threshold_moving=0.001/4 #/2
                    match_threshold=0.5  #0.7
                    align_threshold=0.5
                    image_ransac = 1 # number is specific distance threshold 
                    neighbor_radius0 = 12
                    P1_C3,P2_C3,dist1,dist2,pair_dist_3,fix_spots_3,mov_spots_3,warp_spots_new_3,ICP_affine_3,moving_affine_3,cc_3 = ROI_trackaffine_bash(
                    segmentation1,segmentation2,aa,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
                    ransac_affine,global_affine2,inv_affine2,Transform2,inv_Transform2,Chn,Global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots)
         
                    distances_3 = np.mean(pair_dist_3)
                    lo=np.percentile(np.ndarray.flatten(fixed_ROI_c2),99.5) #Estimate fluorescence intensity of bright DAPI signals
                    bg_dapi=np.percentile(np.ndarray.flatten(fixed_ROI_c2[fixed_ROI_c2!=0]),1) 
                    bg_data=np.percentile(np.ndarray.flatten(fixed_ROI[fixed_ROI!=0]),1) #Estimate background in AF546 channel
                    dapi_factor=np.median((fixed_ROI[fixed_ROI_c2>lo] - bg_data)/(fixed_ROI_c2[fixed_ROI_c2>lo] - bg_dapi)) #Estimate % of signal bleedthrough
                    fixed_ROI  = np.maximum(0, fixed_ROI - bg_data - dapi_factor * (fixed_ROI_c2 -bg_dapi)).astype('float32')#subtract background and bleedthrough                   
                    a_rgb = np.zeros(fixed_ROI.shape + (4,))
                    a_rgb[..., 0] = fixed_ROI.astype(np.uint16)
                    a_rgb[..., 1] = ICP_affine_3.astype(np.uint16)
                    fixed_ROI_c3 = fixed_ROI
                    
                else:
                    #for c0
                    threshold_fixed=0.001/20 #/20
                    threshold_moving=0.001/2 #/2
                    match_threshold=0.5  #0.5
                    align_threshold=0.5  #0.5
                    image_ransac = 1 # number is specific distance threshold for non-dapi track
                    neighbor_radius0 = 12
                    P1_C0,P2_C0,_,_,pair_dist_0,fix_spots_0,mov_spots_0,warp_spots_new_0,fixed_ROI_c0,ICP_affine_0,moving_affine_0,cc_0 = ROI_trackaffine_cca_bash(
                        segmentation1,segmentation2,aa,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,threshold_fixed,threshold_moving,
                        cc_r,match_threshold,align_threshold,ransac_affine,global_affine_fix,inv_affine_fix,Transform_fix,
                        global_affine_mov,inv_affine_mov,Transform_mov,global_affine2,inv_affine2,Transform2,inv_Transform2,
                        Chn,Global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots)
             
                    distances_0 = np.mean(pair_dist_0)
                    a_rgb[..., 2] = fixed_ROI_c0.astype(np.uint16)
                    a_rgb[..., 3] = ICP_affine_0.astype(np.uint16)
#                     a_rgb[..., 3] = moving_affine_0.astype(np.uint16)
                    roi_dir = ROI_RGB_path + '/'+ str(aa) + '_' + 'C3' + Chn +'.tif'
                    imsave(roi_dir, a_rgb)
                    a_rgb = []
            stop
            # save C3 spots, global affine, warp affine.        
            spot_fix3 = fix_spots_3/fix_spacing + [2*cc_3[0],4*cc_3[1],4*cc_3[2]]  # pre:in um, post: in px[um to px + mask(0, in px)]
            spot_fix_c3 = np.column_stack((spot_fix3,np.ones(len(spot_fix3)).dot(aa))) # add ROI_ID
            spot_fix_c3_all = np.row_stack((spot_fix_c3_all,spot_fix_c3))   # for all ROIs
            warp_spots_3 = warp_spots_new_3[:,:3]/fix_spacing + [2*cc_3[0],4*cc_3[1],4*cc_3[2]]  # in pixels of S2
            warp_spots_new_c3 = np.column_stack((warp_spots_3,np.ones(len(warp_spots_new_3)).dot(aa)))
            warp_spots_c3_all = np.row_stack((warp_spots_c3_all,warp_spots_new_c3))
            
            # save C1 spots, global affine, warp affine.
            spot_fix1 = fix_spots_1/fix_spacing + [2*cc_1[0],4*cc_1[1],4*cc_1[2]]  # in phsical distance of S2
            spot_fix_c1 = np.column_stack((spot_fix1,np.ones(len(spot_fix1)).dot(aa)))
            spot_fix_c1_all = np.row_stack((spot_fix_c1_all,spot_fix_c1))
            if warp_spots_new_1.shape[0] == 0:
                warp_spots_c1_all = warp_spots_c1_all
            else:
                warp_spots_1 = warp_spots_new_1[:,:3]/fix_spacing + [2*cc_1[0],4*cc_1[1],4*cc_1[2]]  # in pixels of S2
                warp_spots_new_c1 = np.column_stack((warp_spots_1,np.ones(len(warp_spots_new_1)).dot(aa)))
                warp_spots_c1_all = np.row_stack((warp_spots_c1_all,warp_spots_new_c1))
            
            # save C0 spots, global affine, warp affine.
            spot_fix0 = fix_spots_0/fix_spacing + [2*cc_0[0],4*cc_0[1],4*cc_0[2]]  # in phsical distance of S2
            spot_fix_c0 = np.column_stack((spot_fix0,np.ones(len(spot_fix0)).dot(aa)))
            spot_fix_c0_all = np.row_stack((spot_fix_c0_all,spot_fix_c0))
            warp_spots_0 = warp_spots_new_0[:,:3]/fix_spacing + [2*cc_0[0],4*cc_0[1],4*cc_0[2]]  # in pixels of S2
            warp_spots_new_c0 = np.column_stack((warp_spots_0,np.ones(len(warp_spots_new_0)).dot(aa)))
            warp_spots_c0_all = np.row_stack((warp_spots_c0_all,warp_spots_new_c0))
            
            affine_borrow[j,:] = affine_borrow_idx
            # remember to remove the first 000 coordinates
            Dist_sum_1[j,:] = [aa,ROI_moving,distances_0,P1_C0,P2_C0,
                               distances_1,P1_C1,P2_C1,distances_2,P1_C2,P2_C2,
                               distances_3,P1_C3,P2_C3]
            j = j + 1

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Maxprobe_analysis/R2_R1_3tm50/allroi_matched.csv'

### Export RANSAC result spots 

In [9]:
# A,B,neighbor_radius2 = ROI_trackaffine_cca_bash(
#                         segmentation1,segmentation2,aa,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,threshold_fixed,threshold_moving,
#                         cc_r,match_threshold,align_threshold,ransac_affine,global_affine_fix,inv_affine_fix,Transform_fix,
#                         global_affine_mov,inv_affine_mov,Transform_mov,global_affine2,inv_affine2,Transform2,inv_Transform2,
#                         Chn,Global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots)

# spot_extraction = ['hAir/','RS-FISH/affine_cca/'][1]
# np.savetxt(seg_dir + spot_extraction + '5#_RANSAC_R2_c0_spots.txt',A,delimiter=',')
# np.savetxt(seg_dir + spot_extraction + '5#_RANSAC_R1_c0_spots.txt',B,delimiter=',')

# A,B,neighbor_radius2 = ROI_trackaffine_bash(
#                     segmentation1,segmentation2,aa,ROI_moving,fixed_ROI,moving_ROI,moving_ROI_c2,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
#                     ransac_affine,global_affine2,inv_affine2,Transform2,inv_Transform2,Chn,Global_detection,image_ransac,hAir,neighbor_radius0,Shuffle_spots)

# spot_extraction = ['hAir/','RS-FISH/affine_cca/'][1]
# np.savetxt(seg_dir + spot_extraction + '5#_RANSAC_R2_c3_spots.txt',A,delimiter=',')
# np.savetxt(seg_dir + spot_extraction + '5#_RANSAC_R1_c3_spots.txt',B,delimiter=',')

# Test part for Shiqi

### We use ICP to register point clouds of an example ROI dataset. 
These spots have been corasely registered

In [10]:
neighbor_radius2 =1
viewer = napari.view_image(data.astronaut(), rgb=True)
napari.run()

D:\Anaconda3\envs\FISH\lib\site-packages\napari\_qt\qt_resources\_icons.py:387: UserWarning: Unable to save qt-resources: [Errno 13] Permission denied: 'D:\\Anaconda3\\envs\\FISH\\lib\\site-packages\\napari\\_qt\\qt_resources\\_qt_resources_PyQt5_5_15_2_735669436b5db7ec73a1cc411fb6f05b.py'
  warnings.warn(msg)


In [11]:
#ICP
# if __name__ == "__main__":
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R2_c3_spots.txt'
#     A = np.loadtxt(spotdir, delimiter=',')
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R1_c3_spots.txt'
#     B = np.loadtxt(spotdir, delimiter=',')

if __name__ == "__main__":
    A = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R2_c3_spots.txt',delimiter=',')
    B = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R1_c3_spots.txt',delimiter=',')
    
    Transform,_,_ = icp(A, B,neighbor_radius2)  # A->B 
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist,_,_ = colocalization(A,C,neighbor_radius2*3)  # return in pixel
    
    print(dist)

    viewer.add_points(np.transpose(np.array([A[:,0]/fix_spacing[0],A[:,1]/fix_spacing[1],A[:,2]/fix_spacing[2]])),name ='A', size=1,
                      face_color='green',edge_color='green',blending='opaque') 
    viewer.add_points(np.transpose(np.array([B[:,0]/fix_spacing[0],B[:,1]/fix_spacing[1],B[:,2]/fix_spacing[2]])),name ='B', size=1, 
                      face_color='red',edge_color='red',blending='opaque') 
    viewer.add_points(np.transpose(np.array([C[:,0]/fix_spacing[0],C[:,1]/fix_spacing[1],C[:,2]/fix_spacing[2]])),name ='C', size=1,
                      face_color='yellow',edge_color='yellow',blending='opaque') 

D:\Users\Charlotte\AppData\Local\Temp\ipykernel_22552\4178655402.py:123: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indecies = np.zeros(src.shape[0], dtype=np.int)


No.0 iter icp error:0.4594901660203081
No.1 iter icp error:0.44604218120599903
0.642628220679892


NameError: name 'fix_spacing' is not defined

In [ ]:
#RANSAC ICP 

#if __name__ == "__main__":
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R2_c3_spots.txt'
#     A = np.loadtxt(spotdir, delimiter=',')
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R1_c3_spots.txt'
#     B = np.loadtxt(spotdir, delimiter=',')
    
if __name__ == "__main__":
    A = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R2_c3_spots.txt',delimiter=',')
    B = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R1_c3_spots.txt',delimiter=',')
    
    Transform,_,_ = ICP_RANSAC(A, B,neighbor_radius2)  # A->B 
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist,_,_ = colocalization(A,C,neighbor_radius2*3)  # return in pixel

    print(dist)
    viewer = napari.view_image(data.astronaut(), rgb=True)
    napari.run()
    viewer.add_points(np.transpose(np.array([D[:,0]/fix_spacing[0],D[:,1]/fix_spacing[1],D[:,2]/fix_spacing[2]])),name ='D', size=1,
                      face_color='yellow',edge_color='yellow',blending='opaque') 

In [14]:
#RANSAC ICP 

#if __name__ == "__main__":
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R2_c3_spots.txt'
#     A = np.loadtxt(spotdir, delimiter=',')
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R1_c3_spots.txt'
#     B = np.loadtxt(spotdir, delimiter=',')
    
if __name__ == "__main__":
    A = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R2_c3_spots.txt',delimiter=',')
    B = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R1_c3_spots.txt',delimiter=',')
    
    print(f'icp start')
    Transform_ICP,_,_ = icp(A, B,neighbor_radius2) 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_ICP).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_ICP,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel
   
    print(f'icp ransac start')
    Transform_ICP_RS,_,_ = ICP_RANSAC(A, B,neighbor_radius2)  # A->B 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_ICP_RS).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_ICP_RS,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel
    
    print(f'ransac icp start')
    Transform_RS_ICP,_,_ = RANSAC_ICP(A, B,neighbor_radius2)  # A->B 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_RS_ICP).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_RS_ICP,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel
    
    print(f'ICP distance:{dist_ICP}')
    print(f'ICP_RS distance:{dist_ICP_RS}')
    print(f'RS_ICP distance:{dist_RS_ICP}')
    
#     print(f'ICP:{Transform_ICP}')
#     print(f'ICP_RS:{Transform_ICP_RS}')
#     print(f'RS_ICP:{Transform_RS_ICP}')
#     if (Transform_ICP_RS == Transform).all():
#         print(f'yes ICP_RS=ICP')
#     else:
#         print(f'ICP_RS !=ICP')
#     if (Transform_ICP_RS == Transform_RS_ICP).all():
#         print(f'yes ICP_RS=RS_ICP')
#     else:
#         print(f'ICP_RS !=RS_ICP')

    viewer = napari.view_image(data.astronaut(), rgb=True)
    napari.run()
    viewer.add_points(np.transpose(np.array([D[:,0]/fix_spacing[0],D[:,1]/fix_spacing[1],D[:,2]/fix_spacing[2]])),name ='D', size=1,
                      face_color='yellow',edge_color='yellow',blending='opaque') 

icp start


D:\Users\Charlotte\AppData\Local\Temp\ipykernel_22552\4178655402.py:123: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indecies = np.zeros(src.shape[0], dtype=np.int)


No.0 iter icp error:0.4594901660203081
No.1 iter icp error:0.44604218120599903
icp ransac start
No.0 iter icp error:0.4594901660203081
RSthreshold:0.9037132823659157
100
inliers number:781,error:0.44593923945291863
mean_error_RANSAC: 0.44593923945291863
RANSACpass
No.1 iter icp error:0.46420055824408096
RSthreshold:0.9288345798740351
100
inliers number:779,error:0.44593923945291863
mean_error_RANSAC: 0.44593923945291863
Ignore RS.
distance_iter:[[[0.44593924]
  [0.44593924]]]
ransac icp start
100
inliers number:816,error:0.44593923945291863
Ignore RS.
No.0 iter icp error:0.4594901660203081
RSthreshold:0.9037132823659157
100
inliers number:789,error:0.44593923945291863
Ignore RS.
No.1 iter icp error:0.44604218120599903
RSthreshold:0.9056729171601697
100
inliers number:775,error:0.44593923945291863
Ignore RS.
No.2 iter icp error:0.4459787002434783
distance_iter:[[[0.45949017]
  [0.45949017]]

 [[0.44604218]
  [0.45949017]]

 [[0.4459787 ]
  [0.45949017]]]
ICP distance:0.44593923945291863

NameError: name 'fix_spacing' is not defined

In [ ]:
#RANSAC ICP 

#if __name__ == "__main__":
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R2_c3_spots.txt'
#     A = np.loadtxt(spotdir, delimiter=',')
#     spotdir = seg_dir + 'RS-FISH/affine_cca/5#_RANSAC_R1_c3_spots.txt'
#     B = np.loadtxt(spotdir, delimiter=',')
    
if __name__ == "__main__":
    A = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R2_c0_spots.txt',delimiter=',')
    B = np.loadtxt('D:/Users/Charlotte/OneDrive - International Campus, Zhejiang University/Sternson Lab/seqFISH Project/220717_test files/5#_RANSAC_R1_c0_spots.txt',delimiter=',')
    
    print(f'icp start')
    Transform_ICP,_,_ = icp(A, B,neighbor_radius2) 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_ICP).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_ICP,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel
   
    
    print(f'icp ransac start')
    Transform_ICP_RS,_,_ = ICP_RANSAC(A, B,neighbor_radius2)  # A->B 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_ICP_RS).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_ICP_RS,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel
    
    print(f'ransac icp start')
    Transform_RS_ICP,_,_ = RANSAC_ICP(A, B,neighbor_radius2)  # A->B 
    
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)        
    D = p.dot(np.linalg.inv(Transform_RS_ICP).T)        ## B->A spots (right) with B->A transform (right)
    C = D[:,:3]
    # Test ICP distance change. Minor increased performance
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist_RS_ICP,_,_ = colocalization(A,C,neighbor_radius2)  # return in pixel

    
    print(f'ICP distance:{dist_ICP}')
    print(f'ICP_RS distance:{dist_ICP_RS}')
    print(f'RS_ICP distance:{dist_RS_ICP}')
    
#     print(f'ICP:{Transform_ICP}')
#     print(f'ICP_RS:{Transform_ICP_RS}')
#     print(f'RS_ICP:{Transform_RS_ICP}')
#     if (Transform_ICP_RS == Transform).all():
#         print(f'yes ICP_RS=ICP')
#     else:
#         print(f'ICP_RS !=ICP')
#     if (Transform_ICP_RS == Transform_RS_ICP).all():
#         print(f'yes ICP_RS=RS_ICP')
#     else:
#         print(f'ICP_RS !=RS_ICP')
    

    viewer = napari.view_image(data.astronaut(), rgb=True)
    napari.run()
    viewer.add_points(np.transpose(np.array([D[:,0]/fix_spacing[0],D[:,1]/fix_spacing[1],D[:,2]/fix_spacing[2]])),name ='D', size=1,
                      face_color='yellow',edge_color='yellow',blending='opaque') 